<a href="https://colab.research.google.com/github/Ahmed-Hossam-Aldeen/Beat-detector-using-fourier-analysis/blob/main/beat_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub

In [55]:
import pydub
from pydub import AudioSegment
sound = AudioSegment.from_mp3("output.mp3")
sound.export("output.wav", format="wav")

<_io.BufferedRandom name='output.wav'>

## Full code

In [56]:
import shutil
!mkdir chunks

from pydub import AudioSegment
from pydub.utils import make_chunks
import numpy  as np
from scipy import fft, arange
import librosa
import time

kicks = []
seconds = []
slices = 10

myaudio = AudioSegment.from_file("output.wav" , "wav") 
chunk_length_ms = 1000/slices # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

#Export all of the individual chunks as wav files
def frequency_spectrum(x, sf):
    x = x - np.average(x)  # zero-centering
    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range
    frqarr = frqarr[range(n // 2)]  # one side frequency range
    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]
    return frqarr, abs(x)

for i, chunk in enumerate(chunks):
    chunk_name = "chunk{0}.wav".format(i)
    #print ("exporting", chunk_name)
    chunk.export(f'./chunks/{chunk_name}', format="wav")

    audio = f'./chunks/chunk{i}.wav'
    x, sr = librosa.load(audio)
    frq, X = frequency_spectrum(x, sr)
    
    X = X*10000
    seconds.append(i/slices)
    if X[5]>1000 or x[17]>1000:
        #print("kick detected")
        kicks.append(i/slices)
        
shutil.rmtree("chunks")
print(kicks)
#################################################################################
import cv2
import numpy as np
import matplotlib.pyplot as plt
def cv2_clipped_zoom(img, zoom_factor=0):

    if zoom_factor == 0:
        return img

    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    
    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]
    
    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)
    
    result = cv2.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

img = cv2.imread("test.jpg")
ksize = (5, 5)
scale = 1.04
zoomed = cv2_clipped_zoom(img, scale)
# Using cv2.blur() method 
image = cv2.blur(zoomed, ksize) 
cv2.imwrite("testzoom.jpg",image)
################################################################################
import cv2
frames = []
i = 0

for i in seconds:
  if i in kicks:
    frames.append(cv2.imread('testzoom.jpg'))
    #print(i)
  else:  
    frames.append(cv2.imread('test.jpg'))

height, width, _ = frames[0].shape
out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'DIVX'), slices, (width, height))
[out.write(f) for f in frames]
out.release()
################################################################################
!ffmpeg -i output.mp3 -i video.mp4 -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors=white|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags  +faststart -preset ultrafast videoMono.mp4

mkdir: cannot create directory ‘chunks’: File exists


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: scipy.arange is deprecated and will be removed in SciPy 2.0.0, use numpy.arange instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.


[18.4, 20.6, 21.9, 22.0, 24.2, 24.9, 25.5, 25.6, 27.5, 27.6, 27.8, 29.1, 29.2, 31.3, 31.4, 32.0, 32.1, 32.7, 34.7, 34.9, 35.0, 36.3, 38.5, 39.2, 39.8, 39.9, 41.9, 42.1, 43.4, 43.5, 45.7, 46.3, 46.4, 47.0, 47.6, 47.7, 48.3, 49.0, 49.2, 50.1, 50.5, 50.6, 51.2, 51.6, 51.7, 51.9, 52.8, 53.4, 53.5, 53.9, 54.1, 55.2, 55.3, 55.5, 56.1, 57.0, 57.7, 59.3, 61.9, 62.6, 63.3, 63.5, 64.4, 64.8, 65.5, 66.0, 66.2, 67.1, 68.4, 69.5, 69.8, 73.6, 74.9, 75.1, 89.9, 90.6, 91.3, 91.9, 92.0, 92.1, 92.2, 93.0, 93.1, 93.5, 94.2, 94.6, 94.8, 94.9, 95.7, 95.8, 96.4, 96.8, 96.9, 97.1, 98.2, 98.4, 98.5, 99.1, 100.0, 100.6, 100.7, 101.3, 102.2, 102.3, 107.8, 107.9, 110.1, 110.8, 111.4, 111.5, 113.4, 113.5, 113.7, 115.0, 115.1, 117.3, 117.9, 118.0, 118.6, 120.8, 120.9, 122.2, 124.4, 125.1, 125.7, 125.8, 127.8, 128.0, 129.3, 129.4, 131.6, 132.2, 132.3, 132.9, 133.5, 133.6, 134.2, 134.9, 135.1, 136.0, 136.4, 136.5, 137.1, 137.6, 137.8, 138.7, 139.4, 139.8, 140.0, 141.1, 141.2, 141.4, 142.0, 142.1, 142.9, 143.0, 143.6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

## Functions

In [47]:
import shutil
!mkdir chunks

from pydub import AudioSegment
from pydub.utils import make_chunks
import numpy  as np
from scipy import fft, arange
import librosa
import time

kicks = []
seconds = []
slices = 10

myaudio = AudioSegment.from_file("output.wav" , "wav") 
chunk_length_ms = 1000/slices # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

#Export all of the individual chunks as wav files
def frequency_spectrum(x, sf):
    x = x - np.average(x)  # zero-centering
    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range
    frqarr = frqarr[range(n // 2)]  # one side frequency range
    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]
    return frqarr, abs(x)

for i, chunk in enumerate(chunks):
    chunk_name = "chunk{0}.wav".format(i)
    #print ("exporting", chunk_name)
    chunk.export(f'./chunks/{chunk_name}', format="wav")

    audio = f'./chunks/chunk{i}.wav'
    x, sr = librosa.load(audio)
    frq, X = frequency_spectrum(x, sr)
    
    X = X*10000
    seconds.append(i/slices)
    if X[10]>1000:
        #print("kick detected")
        kicks.append(i/slices)
    #time.sleep(0.1835)    
        
#shutil.rmtree("chunks")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: scipy.arange is deprecated and will be removed in SciPy 2.0.0, use numpy.arange instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.


In [43]:
shutil.rmtree("chunks")

In [ ]:
print(kicks)
print(seconds)

In [ ]:
from IPython.display import Audio
Audio('output.mp3')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
def cv2_clipped_zoom(img, zoom_factor=0):

    if zoom_factor == 0:
        return img

    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    
    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]
    
    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)
    
    result = cv2.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

img = cv2.imread("test.jpg")
ksize = (7, 7)
  
scale = 1.04
zoomed = cv2_clipped_zoom(img, scale)

# Using cv2.blur() method 
image = cv2.blur(zoomed, ksize) 

cv2.imwrite(f"testzoom{scale}.jpg",image)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


True

In [ ]:
import cv2
frames = []
i = 0

for i in seconds:
  if i in kicks:
    frames.append(cv2.imread('testzoom1.04.jpg'))
    #print(i)
  else:  
    frames.append(cv2.imread('test.jpg'))

height, width, _ = frames[0].shape
out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'DIVX'), slices, (width, height))
[out.write(f) for f in frames]
out.release()

In [ ]:
!ffmpeg -i video.mp4 -i output.mp3 -map 0:v -map 1:a -c:v copy -shortest final.mp4

In [ ]:
!ffmpeg -i output.mp3 -i video.mp4 -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors=white|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags  +faststart -preset ultrafast videoMono.mp4

In [ ]:
# Start a webserver in the background, running on port 8000.
%%script bash --bg
python3 -m http.server 8000

Starting job # 3 in a separate thread.


In [ ]:
# Embed the video file.
%%html
<video controls autoplay><source src="http://localhost:8000/videoMono.mp4" type="video/mp4"></video>

In [ ]:
from scipy import fft, arange
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os


def frequency_spectrum(x, sf):
    """
    Derive frequency spectrum of a signal from time domain
    :param x: signal in the time domain
    :param sf: sampling frequency
    :returns frequencies and their content distribution
    """
    x = x - np.average(x)  # zero-centering

    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range

    frqarr = frqarr[range(n // 2)]  # one side frequency range

    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]

    return frqarr, abs(x)


for i in range(0,1000):
  audio = f'./chunks/chunk{i}.wav'
  x, sr = librosa.load(audio)
  frq, X = frequency_spectrum(x, sr)

  X = X*10000

  
  plt.plot(frq, X, 'b')
  plt.xlabel('Freq (Hz)')
  plt.ylabel('|X(freq)|')
  plt.show()
  print("at chunk:",i)
  print(X[4:20])
  #time.sleep(2)

In [38]:
from IPython.display import Audio
Audio('./chunks/chunk220.wav')

In [20]:
for i in range(0,len(X)):
  if X[i] == 1908.670274819701:
    print(i)

4


In [19]:
print(max(X))

1908.670274819701
